In [1]:
import re
import pandas as pd
import numpy as np
import os
import albumentations as A
import cv2

In [2]:
df = pd.read_csv("star_data_3.csv")
df

,Unnamed: 0,Star Name,Distance (pc),Spectral Class,Index,Luminosity Class
0,0,Absolutno,273.845058,G8V,0,V
1,1,Acamar,49.431537,A2V,1,V
2,2,Achernar,42.753313,B6Vpe,2,V
3,3,Achird,5.923041,F9V,3,V
4,4,Acrux,98.716683,B0.5IV+B1V,4,IV
...,...,...,...,...,...,...
426,426,Zibal,36.503011,kA4hA9mA9V,426,V
427,427,Zosma,17.914726,A5IV(n),427,IV
428,428,Zubenelgenubi,23.239600,kA2hA5mA4IV-V,428,IV
429,429,Zubenelhakrabi,50.025013,G8.5III,429,III


In [38]:
stars = df['Star Name'].to_list()
sp = df['Spectral Class'].to_list()
# find_subclass = lambda x: re.search('[0-9]', x).group()
def find_subclass(x):
    res = re.search('[0-9]', x)
    if res:
        return res.group()
    else:
        return None
subclasses = []
# print(len(stars), len(sp))

for i in sp:
    subclasses.append(find_subclass(i))


# stars_subclass = {stars[i]: subclasses[i] for i in range(len(sp))}
# print("Tojil" in stars)
print(len(sp))
print(len(stars))
print(len(subclasses))

for i in subclasses:
    if i is None:
        stars.remove(stars[subclasses.index(i)])
        sp.remove(sp[subclasses.index(i)])
        subclasses.remove(i)
print(len(sp))
print(len(stars))
print(len(subclasses))

stars_to_subclass = {stars[i]: subclasses[i] for i in range(len(stars))}
print(stars_to_subclass)

431
431
431
428
428
428
{'Absolutno': '8', 'Acamar': '2', 'Achernar': '6', 'Achird': '9', 'Acrux': '0', 'Acubens': '7', 'Adhafera': '0', 'Adhara': '1', 'Adhil': '0', 'Ainalrami': '1', 'Aladfar': '2', 'Alasia': '5', 'Albaldah': '2', 'Albali': '9', 'Albireo': '3', 'Alchiba': '1', 'Alcor': '5', 'Alcyone': '7', 'Aldebaran': '5', 'Alderamin': '8', 'Aldhanab': '8', 'Aldhibah': '6', 'Aldulfin': '6', 'Alfirk': '0', 'Algedi': '9', 'Algenib': '2', 'Algieba': '1', 'Algol': '8', 'Algorab': '0', 'Alhena': '1', 'Alioth': '1', 'Aljanah': '0', 'Alkaid': '3', 'Alkalurops': '0', 'Alkaphrah': '1', 'Alkarab': '8', 'Alkes': '0', 'Almaaz': '9', 'Almach': '2', 'Alnair': '6', 'Alnasl': '0', 'Alnilam': '0', 'Alnitak': '9', 'Alniyat': '1', 'Alphard': '3', 'Alphecca': '1', 'Alpheratz': '8', 'Alpherg': '7', 'Alrakis': '6', 'Alrescha': '0', 'Alruba': '2', 'Alsafi': '0', 'Alsciaukat': '4', 'Alsephina': '2', 'Alshain': '8', 'Alshat': '9', 'Altair': '7', 'Altais': '9', 'Alterf': '4', 'Aludra': '5', 'Alula Australis':

In [40]:
for i in os.listdir("/Users/jminding/Documents/Research/Star Images 3"):
    if i.replace(".jpg", "").replace(".jpeg", "") not in stars:
        print(i)
        os.remove("/Users/jminding/Documents/Research/Star Images 3/" + i)

Mizar.jpg
Lich.jpg
Malmok.jpg
Lerna.jpg
Atakoraka.jpg
Taika.jpg
Muspelheim.jpg
Naledi.jpg
Amansinaya.jpg
Dabih.jpg
Tevel.jpg
Diya.jpg
Geminga.jpg
Parumleo.jpg
Acrab.jpg
Tojil.jpg
HAT-P-21.jpg
Marsic.jpg
Pipoltr.jpg
Horna.jpg
Alya.jpg
Markeb.jpg
Tuiren.jpg


In [41]:
for i in os.listdir("/Users/jminding/Documents/Research/Star Images 3"):
    star_name = i.replace(".jpg", "").replace(".jpeg", "")
    subdir = subclasses[stars.index(star_name)]
    if not os.path.exists("/Users/jminding/Documents/Research/Star Images 3/" + subdir):
        os.makedirs("/Users/jminding/Documents/Research/Star Images 3/" + subdir)
    os.rename("/Users/jminding/Documents/Research/Star Images 3/" + i, "/Users/jminding/Documents/Research/Star Images 3/" + subdir + "/" + i)

In [9]:
import os
from PIL import Image, ImageOps
import itertools

def apply_transformations(image_path, output_dir):
    image = Image.open(image_path)

    transformations = [
        ("H", lambda img: img.transpose(Image.FLIP_LEFT_RIGHT)),
        ("V", lambda img: img.transpose(Image.FLIP_TOP_BOTTOM)),
        ("R90", lambda img: img.rotate(90, expand=True)),
        ("R180", lambda img: img.rotate(180, expand=True)),
        ("R270", lambda img: img.rotate(270, expand=True))
    ]

    combinations = []
    for r in range(1, len(transformations) + 1):
        combinations.extend(itertools.combinations(transformations, r))

    for combination in combinations:
        transformed_image = image.copy()
        description = '_'.join([abbr for abbr, _ in combination])
        for _, transform in combination:
            transformed_image = transform(transformed_image)
        filename = os.path.basename(image_path)
        new_filename = f"transformed_{description}_{filename}"
        output_path = os.path.join(output_dir, new_filename)
        transformed_image.save(output_path)
    image.close()

def process_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                file_path = os.path.join(root, file)
                apply_transformations(file_path, root)

if __name__ == "__main__":
    directory = "/Users/jminding/Documents/Research/Star Images 3"
    process_directory(directory)
    print("Transformation complete.")


Transformation complete.
